In [ ]:
import os

# NOTE: Change the working directory to /workspace, in case you do use jupyter notebooks without vscode,
# respectivley you did not set "jupyter.notebookFileRoot": "${fileDirname}/workspace" in your devcontainer.json.
# This is realted to the issue how Jupyter Notebook handles the working directory.
# Jupyter Notebook sets its own working directory when it starts up to the root dir of the container.

# os.chdir('/workspace')

In [1]:
!pwd

/workspace


In [ ]:
from torch import optim, nn, utils, Tensor, rand
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning as L

In [5]:
# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))


# define the LightningModule
class LitAutoEncoder(L.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

In [6]:
# setup data
dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)

100%|██████████| 9912422/9912422 [00:00<00:00, 10186916.93it/s]


Extracting /workspace/MNIST/raw/train-images-idx3-ubyte.gz to /workspace/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 3119079.58it/s]

Extracting /workspace/MNIST/raw/train-labels-idx1-ubyte.gz to /workspace/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 20245287.38it/s]


Extracting /workspace/MNIST/raw/t10k-images-idx3-ubyte.gz to /workspace/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 260991.17it/s]

Extracting /workspace/MNIST/raw/t10k-labels-idx1-ubyte.gz to /workspace/MNIST/raw



In [7]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = L.Trainer(limit_train_batches=100, max_epochs=1)
trainer.fit(model=autoencoder, train_dataloaders=train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/conda/envs/env1/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Missing logger folder: /workspace/lightning_logs

  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 50.4 K
1 | decoder | Sequential | 51.2 K
---------------------------------------
101 K     Trainable params
0         Non-trainable params
101 K     Total params
0.407 

Epoch 0: 100%|██████████| 100/100 [00:24<00:00,  4.06it/s, v_num=0]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 100/100 [00:24<00:00,  4.05it/s, v_num=0]


In [8]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

# embed 4 fake images!
fake_image_batch = rand(4, 28 * 28, device=autoencoder.device)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡ 
Predictions (4 image embeddings):
 tensor([[-0.1278,  0.5242,  0.4316],
        [-0.2679,  0.5131,  0.4414],
        [-0.1504,  0.4849,  0.3815],
        [-0.2395,  0.6004,  0.4601]], grad_fn=<AddmmBackward0>) 
 ⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡⚡
